# XED data

In [41]:
import polars as pl
import os 
# get all files per country
path = "emotion-data/XED"
files = os.listdir(path)

In [42]:
df = pl.DataFrame()
schema = {"text":pl.String,
          "labels":pl.String,
         "language":pl.String}
for f in files:
    tmp = pl.read_csv(path+"/"+f, separator="\t",schema=schema, ignore_errors=True)
    lang = f.split("-")[0]
    tmp = tmp.with_columns(language=pl.Series([lang]*len(tmp)))
    if tmp.is_empty():
        df=tmp
    else:
        df=pl.concat([df,tmp],how="vertical")

In [43]:
df.info()

AttributeError: 'DataFrame' object has no attribute 'info'

In [25]:
df["labels"].unique()
df

text,labels,language
str,str,str
"""Da nam nisu ucjenjene glave, z…","""4""","""hr"""
"""Zašto ne popijemo piće za dobr…","""2, 5""","""hr"""
"""Neće moći.""","""3""","""hr"""
"""Ovdje više tako ne ide Kid.""","""6""","""hr"""
"""Nikom nije stalo.""","""1, 3""","""hr"""
…,…,…
"""Ez da ezer gertatzen, David.""","""5, 6, 8""","""eu"""
"""Pentsatuko zenuela zin egin du…","""1, 6""","""eu"""
"""Tonia!""","""5, 7""","""eu"""


In [26]:
# exclude languages not present in parlamint 4.0 dataset
# language codes by ISO 639 in parlamint
lang_codes = ["bs","bg","hr","cs","da","nl","en","et","fi","fr","de",
                 "hu","is","it","lv","el","no","pl","pt","ru","sr","sl",
                 "es","sv","tr","uk"]

# exclude langs
df = df.filter(pl.col("language").is_in(lang_codes))

In [50]:
import pandas as pd

df = pd.DataFrame(df)

df.columns = ['text', 'labels', 'lang']

In [51]:
# convert string labels to list and recode from 0 to n-1
# df = df.with_columns(pl.col("labels").map_elements(lambda s: [[int(x)-1] for x in s.split(",") if x.isdigit]))
import numpy as np

def to_array(string):
    string = string.replace(" ", "")
    number_list = string.split(',')
    number_list = [int(num) for num in number_list]
    number_array = np.array(number_list)
    
    return number_array

df.labels = df['labels'].apply(to_array)


In [54]:
from sklearn.preprocessing import MultiLabelBinarizer


def one_hot_encoding(ds):
    mlb = MultiLabelBinarizer()
    ds = ds.join(pd.DataFrame(mlb.fit_transform(ds.pop('labels')),
                          columns=mlb.classes_,
                          index=ds.index))
    
    return ds

df = one_hot_encoding(df)



In [57]:
df = pl.DataFrame(df)

text,lang,1,2,3,4,5,6,7,8
str,str,i64,i64,i64,i64,i64,i64,i64,i64
"""Da nam nisu ucjenjene glave, z…","""hr""",0,0,0,1,0,0,0,0
"""Zašto ne popijemo piće za dobr…","""hr""",0,1,0,0,1,0,0,0
"""Neće moći.""","""hr""",0,0,1,0,0,0,0,0
"""Ovdje više tako ne ide Kid.""","""hr""",0,0,0,0,0,1,0,0
"""Nikom nije stalo.""","""hr""",1,0,1,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…
"""Ez da ezer gertatzen, David.""","""eu""",0,0,0,0,1,1,0,1
"""Pentsatuko zenuela zin egin du…","""eu""",1,0,0,0,0,1,0,0
"""Tonia!""","""eu""",0,0,0,0,1,0,1,0


In [60]:
# save full dataset
df.write_parquet("data.parquet")

In [73]:
df = df.with_columns(pl.col('text').map_elements(lambda x: len(x)).alias('len_text'))
# Compute the mean length
#mean_length = df.select(pl.col('StringLength').mean())
df['len_text'].mean()

/tmp/ipykernel_26079/2385366718.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(pl.col('text').map_elements(lambda x: len(x)).alias('len_text'))


36.490692998826404